<a href="https://colab.research.google.com/github/prathithbhargav/AlphaMut/blob/master/3_inference_of_Helix-in-protein_trained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [1]:
%%capture
#@title Installing Required Packages
!pip install stable-baselines3
!pip install biotite==0.41
!pip install biopandas
!pip install py3dmol

In [2]:
%%capture
!unzip whole_protein_utils.zip

In [13]:
import whole_protein_utils
import pandas as pd
import numpy as np
import glob, os, shutil, pickle
import gymnasium as gym
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3 import PPO

In [14]:

def evaluate_model(estimator, env, num_episodes):
    total_reward_episode = [0] * num_episodes
    actions_taken_in_episodes = {}
    number_of_mutations_per_episode = [0] * num_episodes
    file_chosen_for_mutations = {}
    for episode in tqdm(range(num_episodes)):
        actions_taken_in_episodes[episode] = []
        file_chosen_for_mutations[episode] = [0]
        rewards = []

        state, info = env.reset()
        while True:
            action= estimator.predict(state)[0]
            next_state, reward, terminated, truncated, info = env.step(action)
            total_reward_episode[episode] += reward
            rewards.append(reward)
            name_of_the_protein = os.path.basename(env.path_of_template_pdb_file),
            actions_taken_in_episodes[episode].append([info['amino_acid_position'],info['new_amino_acid']])
            number_of_mutations_per_episode[episode]+=1
            file_chosen_for_mutations[episode][0] = str(name_of_the_protein[0])
            if terminated or truncated:
                break
            state = next_state
#         print(f'{episode} episode done')
    return total_reward_episode,np.mean(total_reward_episode),np.std(total_reward_episode),actions_taken_in_episodes,number_of_mutations_per_episode,file_chosen_for_mutations


In [15]:
def validate_model_disease(saved_model_file_path,csv_file_path,maximum_mutations_allowed,number_of_iterations):
    base_name_of_saved_model = os.path.basename(saved_model_file_path)
    # creating the folder to save structures
    number_of_episodes_to_run_validation = number_of_iterations
    csv_file_base_path = os.path.basename(csv_file_path).split('.')[0]
    base_name_of_saved_model = base_name_of_saved_model
    os.makedirs(f'disease_relevance/structures_obtained_disease_{csv_file_base_path}_5_only',exist_ok=True)
    algorithm_used_for_training = 'PPO'

    env = whole_protein_utils.ProteinEvolution(file_containing_sequence_database=csv_file_path,
                                        protein_length_limit=250,
                                          folder_to_save_validation_files=f'disease_relevance/structures_obtained_disease_{csv_file_base_path}_5_only',
                                          reward_cutoff=30,
                                          unique_path_to_give_for_file='validation_try',
                                          sequence_encoding_type='esm',
                                          maximum_number_of_allowed_mutations_per_episode=maximum_mutations_allowed,
                                          use_proline=False,
                                               use_plddt_in_reward=False,
                                          validation=True)

    folder_for_saving_stuff = f'disrupted_helices/results_obtained_{csv_file_base_path}_{maximum_mutations_allowed}_only'
    os.makedirs(folder_for_saving_stuff,exist_ok=True)
    loaded_estimator = PPO.load(saved_model_file_path)


    total_reward_array,mean_validation_reward, standard_deviation,actions_taken, mutations_array, files_chosen_for_mutations = evaluate_model(estimator=loaded_estimator, env=env,num_episodes=number_of_episodes_to_run_validation)
    print(mutations_array)


In [16]:
import warnings
warnings.filterwarnings("ignore")
validate_model_disease('/content/training_whole_protein_PPO_embedding_esm_length_limit_250_gamma_0.99_ent_factor_0.0_maximum_mutations_13_use_plddt_False_use_proline_False',
                       csv_file_path='myoglobin.csv',
                       maximum_mutations_allowed=7,
                       number_of_iterations=10)

  0%|          | 0/10 [00:00<?, ?it/s]

82
82


KeyboardInterrupt: 